In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 47.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=003a5e21e4a16fc781a2d984ce9d7be3526f25381efff177d91227f899677fc0
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Logistic Regression Model").getOrCreate()
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.sql import functions as f
import time

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()
df = spark.read.csv('/content/drive/MyDrive/Datasets/Thesis/New Dataset/dr16.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- modelMag_u: double (nullable = true)
 |-- modelMag_g: double (nullable = true)
 |-- modelMag_r: double (nullable = true)
 |-- modelMag_i: double (nullable = true)
 |-- modelMag_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- mmug: double (nullable = true)
 |-- mmgr: double (nullable = true)
 |-- mmri: double (nullable = true)
 |-- mmiz: double (nullable = true)
 |-- mfug: double (nullable = true)
 |-- mfgr: double (nullable = true)
 |-- mfri: do

In [ ]:
import pyspark.sql.functions as func
df = df.withColumn("redshift", func.round(df["redshift"], 2).cast('integer'))
df.groupBy('redshift').count().show()

+--------+------+
|redshift| count|
+--------+------+
|       1| 48930|
|       3| 71008|
|       2| 31638|
|       0|409778|
|       6|  5347|
|       5|  6185|
|       4|  9264|
|       7|    81|
+--------+------+



In [ ]:
df = df.drop('Unnamed: 0', '_c0')
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|petroR90_z|       r|       i|       z|     mmug|     mmgr|     mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|redshift|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+
|   23.2391|  21.44343|  19.67306|    18.944|  18.55688|  23.69614|  22.34922|  20.59603|  19.86871|  19.45047|  1.306018|  3.220677|  0.761024|  1.247238|19.67306|  18.944|18.55688|22

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("objid", monotonically_increasing_id())
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|petroR90_z|       r|       i|       z|     mmug|     mmgr|     mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|redshift|objid|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+
|   23.2391|  21.44343|  19.67306|    18.944|  18.55688|  23.69614|  22.34922|  20.59603|  19.86871|  19.45047|  1.306018|  3.220677|  0.761024|  1.247238|19.67306|  

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 
                                         'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 
                                         'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz'], outputCol = 'features')

In [ ]:
output = assembler.transform(df)
final_df = output.select('features', 'redshift')
final_df.show(3)

+--------------------+--------+
|            features|redshift|
+--------------------+--------+
|[23.2391,21.44343...|       0|
|[24.05348,21.2472...|       0|
|[26.32478,20.7405...|       0|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
classifier = LogisticRegression(labelCol = 'redshift', featuresCol = 'features')

In [ ]:
start_time = time.time()
model = classifier.fit(train)
print("%s seconds" % (time.time() - start_time))

72.29490375518799 seconds


In [ ]:
pred_data = model.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(metricName = 'accuracy',labelCol = 'redshift')
AUC = evaluator.evaluate(pred_data)

In [ ]:
print(AUC)

0.8692868157507037


In [ ]:
print('test data (weightedPrecision): ', evaluator.setMetricName('weightedPrecision').evaluate(pred_data))
print('test data (weightedRecall): ', evaluator.setMetricName('weightedRecall').evaluate(pred_data))

test data (weightedPrecision):  0.8507842597215167
test data (weightedRecall):  0.8692868157507037


#Cascading

In [ ]:
pred_data2 = model.transform(final_df)

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
pred_data2 = pred_data2.withColumn("objid", monotonically_increasing_id())
pred_data2.show(5)

+--------------------+--------+--------------------+--------------------+----------+-----+
|            features|redshift|       rawPrediction|         probability|prediction|objid|
+--------------------+--------+--------------------+--------------------+----------+-----+
|[23.2391,21.44343...|       0|[5.87059611922958...|[0.96773116381394...|       0.0|    0|
|[24.05348,21.2472...|       0|[5.88588397079877...|[0.96092800263189...|       0.0|    1|
|[26.32478,20.7405...|       0|[6.85893430264888...|[0.98355649310840...|       0.0|    2|
|[24.76801,19.3207...|       0|[8.32904498837645...|[0.99753310998625...|       0.0|    3|
|[21.75992,19.0856...|       0|[9.20951513224886...|[0.99927006959495...|       0.0|    4|
+--------------------+--------+--------------------+--------------------+----------+-----+
only showing top 5 rows



In [ ]:
print(df.count(), len(df.columns))
print(pred_data2.count(), len(pred_data2.columns))

new_df = df.join(pred_data2, df.objid == pred_data2.objid).select(df["*"],pred_data2["prediction"])
#new_df = df.join(dt_predictions, df.objid == dt_predictions.objid)
new_df.show(5)

582231 27
582231 6
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+----------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|petroR90_z|       r|       i|       z|     mmug|     mmgr|     mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|redshift|objid|prediction|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+----------+
|   23.2391|  21.44343|  19.67306|    18.944|  18.55688|  23.69614|  22.34922|  20.59603|  19.86871|  19.45047|  1

In [ ]:
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 
                                         'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 
                                         'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz', 'prediction'], outputCol = 'features2')

In [ ]:
output = assembler.transform(new_df)
final_df = output.select('features2', 'redshift')
final_df.show(3)

+--------------------+--------+
|           features2|redshift|
+--------------------+--------+
|[23.2391,21.44343...|       0|
|[24.05348,21.2472...|       0|
|[25.42423,21.6029...|       0|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])
classifier = LogisticRegression(labelCol = 'redshift',featuresCol = 'features2')

In [ ]:
model2 = classifier.fit(train)

In [ ]:
pred_data2 = model2.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(metricName = 'accuracy',labelCol = 'redshift')
AUC = evaluator.evaluate(pred_data2)
print(AUC)

0.8688710930360081


In [ ]:
print('test data (weightedPrecision): ', evaluator.setMetricName('weightedPrecision').evaluate(pred_data2))
print('test data (weightedRecall): ', evaluator.setMetricName('weightedRecall').evaluate(pred_data2))

test data (weightedPrecision):  0.8497239328861631
test data (weightedRecall):  0.8688710930360082
